In [1]:
import numpy as np
import open3d as o3d
from tqdm import tqdm_notebook
import os, sys, shutil
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [2]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/"

FOLDER = "d"
# OUTPUT_FOLDER = "d_img_40_512"
IS_MASK = False

# STATIC_PATH  = os.path.join(BASE_PATH, STATIC_FOLDER)
FOLDER_PATH = os.path.join(BASE_PATH, FOLDER)
# OUTPUT_FOLDER_PATH = os.path.join(BASE_PATH, OUTPUT_FOLDER)
# if not os.path.exists(OUTPUT_FOLDER_PATH):
#     os.makedirs(OUTPUT_FOLDER_PATH)
# else:
#     shutil.rmtree(OUTPUT_FOLDER_PATH)
#     os.makedirs(OUTPUT_FOLDER_PATH)

In [3]:
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def linear_transform(sample_img, A=-1, B=1, a=0, b=255):
    mul_fact = (b-a)/(B-A)
    sub_fact = A
    add_fact = a
    out_img = (sample_img-sub_fact)*mul_fact + add_fact
    return out_img

def get_rgb_from_img(img, is_mask=IS_MASK):
    shp = img.shape
    if not is_mask:
        img = img.reshape(1, shp[0], shp[1], shp[2])
        frame_untransformed = from_polar_np(img)[0]
    else:
        replicate = img[1].reshape(1,shp[1], shp[2]) # dim 1 is dynamic in the mask npy
        frame_untransformed = np.concatenate([replicate, replicate, replicate])
        
    frame = linear_transform(frame_untransformed)
    frame_img = frame.transpose(1,2,0).astype("uint8")
    return frame_img

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def getint(name):
    try:
        return int(name.split('.')[0])
    except Exception as e:
        print("Error occured while trying to read {}".format(name))
    return None

def get_pcd(idx, pcd_arr, mask_arr=None):
    this_arr = pcd_arr[idx].reshape(3,-1).T
    this_pcd = o3d.geometry.PointCloud()
    this_pcd.points = o3d.utility.Vector3dVector(this_arr)

    if mask_arr is not None:
        single_clr_arr = mask_arr[idx,1].reshape(1,-1)
        clr_arr = np.concatenate([single_clr_arr, np.zeros(single_clr_arr.shape), np.zeros(single_clr_arr.shape)],axis=0).T
        this_pcd.colors = o3d.utility.Vector3dVector(clr_arr)
    return this_pcd

1. Load npy

In [4]:
npy_file = sorted(os.listdir(FOLDER_PATH), key=getint)[0]
print(npy_file)
npy_path = os.path.join(FOLDER_PATH, npy_file)
# npy_path = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/rs/emptycities_out.npy"
npy_arr = np.load(npy_path)
npy_arr.shape

Error occured while trying to read emptycities_testd15.npy
emptycities_testd15.npy


(1274, 2, 64, 1024)

2. Load one image from npy

In [5]:
img_arr = npy_arr[10]
img_arr.shape

(2, 64, 1024)

In [6]:
temp_arr = from_polar_np(img_arr.reshape(1,img_arr.shape[0],img_arr.shape[1],img_arr.shape[2]))[0].reshape(3,-1).T
temp_arr.shape

temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

3. Change to 40,512

In [7]:
img_40_512_arr = img_arr[:,5:45,::2]
img_40_512_arr.shape

(2, 40, 512)

In [8]:
temp_arr = from_polar_np(img_40_512_arr.reshape(1,img_40_512_arr.shape[0],img_40_512_arr.shape[1],img_40_512_arr.shape[2]))[0].reshape(3,-1).T
temp_arr.shape

temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

4 Change to 3d

In [9]:
shp = img_40_512_arr.shape
img_unsqueeze = img_40_512_arr.reshape(1, shp[0], shp[1], shp[2])
img_3_40_512_arr = from_polar_np(img_unsqueeze)[0]
img_3_40_512_arr.shape

(3, 40, 512)

In [10]:
temp_arr = img_3_40_512_arr.reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

5 Apply linear transform

In [11]:
img_transformed = linear_transform(img_3_40_512_arr, A=-1, B=1, a=0, b=255)
img_transformed.shape

(3, 40, 512)

In [12]:
pd.DataFrame(img_3_40_512_arr.reshape(1,-1)[0]).describe()

,0
count,61440.000000
mean,0.005813
std,0.052854
min,-0.195008
25%,-0.029010
50%,0.019917
75%,0.028146
max,0.211825


In [13]:
pd.DataFrame(img_transformed.reshape(1,-1)[0]).describe()

,0
count,61440.000000
mean,128.227020
std,6.738495
min,102.636520
25%,123.801287
50%,130.039368
75%,131.088558
max,154.507675


In [15]:
temp_arr = img_transformed.reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

In [16]:
temp_arr = linear_transform(img_transformed, A=0, B=255, a=-1, b=1).reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

transformation checks!

6 tranpose

In [17]:
img_transformed_transposed = img_transformed.transpose(1,2,0)
img_transformed_transposed.shape

(40, 512, 3)

In [18]:
temp_arr = linear_transform(img_transformed_transposed.transpose(2,0,1), A=0, B=255, a=-1, b=1).reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

7 Convert to uint8

In [21]:
# frame_img = img_transformed_transposed.astype("uint8")
frame_img = np.round(img_transformed_transposed)
frame_img.shape

(40, 512, 3)

In [22]:
temp_arr = linear_transform(frame_img.transpose(2,0,1), A=0, B=255, a=-1, b=1).reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

8 Save image

In [14]:
plt.imsave("temp.png", np.round(frame_img))

NameError: name 'frame_img' is not defined

In [23]:
temp2_arr = cv2.imread("temp.png")

temp_arr = linear_transform(temp2_arr.transpose(2,0,1), A=0, B=255, a=-1, b=1).reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

What is diff between temp2_arr and frame_img array?

In [ ]:
frame_img.shape, temp2_arr.shape

In [ ]:
frame_img[:,:,0], frame_img[:,:,1], frame_img[:,:,2]

In [ ]:
temp2_arr[:,:,0], temp2_arr[:,:,1], temp2_arr[:,:,2]

In [ ]:
temp3_arr = np.concatenate([temp2_arr[:,:,2:3], temp2_arr[:,:,1:2], temp2_arr[:,:,0:1]], axis=-1)
temp3_arr.shape

In [ ]:
set((frame_img == temp3_arr).reshape(1,-1)[0])

In [ ]:
frame = linear_transform(frame_untransformed)
frame_img = frame.transpose(1,2,0).astype("uint8")

In [ ]:

for npy_file in sorted(os.listdir(FOLDER_PATH), key=getint):
    print(npy_file)
    npy_path = os.path.join(FOLDER_PATH, npy_file)
    npy_arr = np.load(npy_path)
    
    file_idx = 1
    OUTPUT_sub_folder_PATH = os.path.join(OUTPUT_FOLDER_PATH, npy_file.split(".")[0])
    if not os.path.exists(OUTPUT_sub_folder_PATH):
        os.makedirs(OUTPUT_sub_folder_PATH)
    else:
        shutil.rmtree(OUTPUT_sub_folder_PATH)
        os.makedirs(OUTPUT_sub_folder_PATH)
    
    for arr in tqdm_notebook(npy_arr):
        this_arr = get_rgb_from_img(arr[:,5:45,::2])
        
        png_file = str(file_idx) + ".png"
        png_path = os.path.join(OUTPUT_sub_folder_PATH, png_file)
        plt.imsave(png_path, this_arr)
        
        file_idx += 1

In [ ]:
this_arr.shape